In [1]:
from base_classification import *

In [2]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm
# for sampling
import random # use random.choice? and random.sample for interactions

from sklearn.metrics import make_scorer
from sklearn.datasets import make_classification
from sklearn.metrics import *


In [3]:
X, y = make_classification(2000)

In [4]:
from sklearn import datasets
iris = datasets.load_iris()

In [5]:
def eval_pipeline(additional_feats, X, y, verbose=True):
    #print(additional_feats)    
    pipeline = additional_feats[:]
    pipeline.append(('SVC', SVC()))
    model = Pipeline(pipeline[:])

    # split data into 10 folds
    kfold = KFold(n_splits=10, shuffle=False, random_state=42)
    results = cross_val_score(model, X, y, cv=kfold)
    if verbose:
        print("Result: {}".format(results.mean()))
    return results.mean()

In [6]:
# take a random sample
eval_pipeline([], X, y)

Result: 0.861


0.86099999999999999

In [7]:
#### do some iterations

In [8]:
bmars = BMARS(X)

In [9]:
bmars.construct_pipeline(False)

[('union',
  FeatureUnion(n_jobs=1, transformer_list=[('base model', BaseModel())],
         transformer_weights=None))]

In [10]:
# evaluate...
eval_pipeline(bmars.construct_pipeline(False), X, y)

Result: 0.861


0.86099999999999999

In [11]:
# one iteration of mh
bk, dk, ck = output_prob_state(0)
u = np.random.uniform()
action = output_action(u, bk, dk, ck)

In [12]:
l = 20.62195407652395

In [13]:
basis = bmars.perform_action(action)

In [14]:
proposed_model = BMARS(**bmars.export())

In [15]:
output = bmars_sample_basis(X, list(basis))
output

{'basis': [10], 'knot': array([ 0.21158865]), 'sign': [1]}

In [16]:
if action == 'birth':
    output = bmars_sample_basis(X, list(basis), {'signs':[-1, 1]})

In [17]:
# do other things for birth
proposed_model.add_basis(**output)

In [18]:
# evaluate...
eval_pipeline( proposed_model.construct_pipeline(False), X, y)

Result: 0.859


0.85899999999999999

In [19]:
interaction = 2

In [20]:
output_prob_state(30+1, l)

(0.25777442595655153, 0.4, 0.34222557404344844)

In [21]:
accept_proposal_ratio(X, y, l, interaction, bmars, proposed_model, mode='birth')

122199.86479694321

In [22]:
accept_proposal_ratio(X, y, l, interaction, proposed_model, bmars, mode='death')

8.1833151097317303e-06

In [23]:
acceptance_proba(X, y, l, interaction, bmars, proposed_model, mode='birth')

[0 1 1 ..., 1 0 1]
[0 1 1 ..., 1 0 1]
bayes_ratio: 1.04098427776
prior_ratio: 2.45499453292e-05
proposal_ratio: 122199.864797


1.0

In [24]:
acceptance_proba(X, y, l, interaction, proposed_model, bmars, mode='death')

[0 1 1 ..., 1 0 1]
[0 1 1 ..., 1 0 1]
bayes_ratio: 1.02074536442
prior_ratio: 40733.2882656
proposal_ratio: 8.18331510973e-06


0.34024845480692745

In [25]:
# simulate a few runs...

u = np.random.uniform()
u

0.7877899997849193

In [26]:
current_basis = proposed_model.export()['basis']
k = len(current_basis)+1
bk, dk, ck = output_prob_state(k)

In [27]:
action = output_action(np.random.uniform(), bk, dk, ck)
action = 'birth'

In [28]:
basis = proposed_model.perform_action(action)

In [29]:
list(basis)

[9, 13]

In [30]:
output = bmars_sample_basis(X, list(basis), {'signs':[-1, 1]})

In [31]:
proposed_model1 = BMARS(**proposed_model.export())
proposed_model1.add_basis(**output)

In [32]:
eval_pipeline(proposed_model.construct_pipeline(False), X, y)

Result: 0.859


0.85899999999999999

In [33]:
eval_pipeline(proposed_model1.construct_pipeline(False), X, y)

Result: 0.858


0.85799999999999998

In [35]:
acceptance_proba(X, y, l, interaction, proposed_model, proposed_model1, mode=action)

[0 1 1 ..., 1 0 1]
[0 1 1 ..., 1 0 1]
bayes_ratio: 0.969389823637
prior_ratio: 3.06874316615e-09
proposal_ratio: 325866306.125


0.96938982363673232